In [1]:
import pandas as pd
import numpy as np
import datetime
from itertools import compress

In [2]:
DATOS_TRANSACCIONALES_1 = pd.read_parquet('sample_data_0006_part_00.parquet', engine='pyarrow')

In [122]:
DATOS_TRANSACCIONALES_1.shape

(10758418, 8)

In [4]:
DATOS_TRANSACCIONALES_2 = pd.read_parquet('sample_data_0007_part_00.parquet', engine='pyarrow')

In [35]:
np.timedelta64(DATOS_TRANSACCIONALES_1['transaction_date'][1] - DATOS_TRANSACCIONALES_1['transaction_date'][1000000], 'h' )
#(df.fr-df.to).astype('timedelta64[h]')

numpy.timedelta64(2935,'h')

In [162]:
## SUMA Y CONTEO 24HORAS
DATOS_TRANSACCIONALES_1['suma_trx'] = DATOS_TRANSACCIONALES_1.groupby([pd.Grouper(key='transaction_date', freq='24h'), 'user_id'])['transaction_amount'].transform('sum')
DATOS_TRANSACCIONALES_1['count_trx'] = DATOS_TRANSACCIONALES_1.groupby([pd.Grouper(key='transaction_date', freq='24h'), 'user_id'])['transaction_amount'].transform('count')

In [165]:
## SUMAR POR CREDITO Y DEBITO
DATOS_TRANSACCIONALES_1['suma_trx_debito'] = DATOS_TRANSACCIONALES_1[DATOS_TRANSACCIONALES_1['transaction_type'] == 'DEBITO'].groupby([pd.Grouper(key='transaction_date', freq='24h'), 'user_id'])['transaction_amount'].transform(np.sum)
DATOS_TRANSACCIONALES_1['suma_trx_credito'] = DATOS_TRANSACCIONALES_1[DATOS_TRANSACCIONALES_1['transaction_type'] == 'CREDITO'].groupby([pd.Grouper(key='transaction_date', freq='24h'), 'user_id'])['transaction_amount'].transform(np.sum)
#DATOS_TRANSACCIONALES_1['conteo_trx'] =DATOS_TRANSACCIONALES_1.groupby([pd.Grouper(key='transaction_date', freq='24h'), 'user_id','transaction_type'])['transaction_amount'].transform('count')

In [166]:
## CONTEO POR CREDITO Y DEBITO
DATOS_TRANSACCIONALES_1['cuenta_trx_debito'] = DATOS_TRANSACCIONALES_1[DATOS_TRANSACCIONALES_1['transaction_type'] == 'DEBITO'].groupby([pd.Grouper(key='transaction_date', freq='24h'), 'user_id'])['transaction_amount'].transform('count')
DATOS_TRANSACCIONALES_1['cuenta_trx_credito'] = DATOS_TRANSACCIONALES_1[DATOS_TRANSACCIONALES_1['transaction_type'] == 'CREDITO'].groupby([pd.Grouper(key='transaction_date', freq='24h'), 'user_id'])['transaction_amount'].transform('count')

In [167]:
## CONTAR LAS CUENTAS DIFERENTES POR DEBITO Y CREDITO
DATOS_TRANSACCIONALES_1['count_cuentas_debito'] = DATOS_TRANSACCIONALES_1[DATOS_TRANSACCIONALES_1['transaction_type'] == 'DEBITO'].groupby([pd.Grouper(key='transaction_date', freq='24h'), 'user_id','transaction_type'])['account_number'].transform('nunique')
DATOS_TRANSACCIONALES_1['count_cuentas_credito'] = DATOS_TRANSACCIONALES_1[DATOS_TRANSACCIONALES_1['transaction_type'] == 'CREDITO'].groupby([pd.Grouper(key='transaction_date', freq='24h'), 'user_id','transaction_type'])['account_number'].transform('nunique')

In [168]:
DATOS_TRANSACCIONALES_1 = DATOS_TRANSACCIONALES_1.fillna(0)

In [144]:
DATOS_TRANSACCIONALES_1.head()

,merchant_id,_id,subsidiary,transaction_date,account_number,user_id,transaction_amount,transaction_type,suma_trx_debito,suma_trx_credito,cuenta_trx_debito,cuenta_trx_credito,count_cuentas_debito,count_cuentas_credito,DEBITO=CREDITO,suma_trx,count_trx
0,075d178871d8d48502bf1f54887e52fe,aa8dacff663072244d0a8ab6bbe36b93,824b2af470cbe6a65b15650e03b740fc,2021-09-12 18:32:03,648e257c9d74909a1f61c54b93a9e1b3,ba42d192a145583ba8e7bf04875f837f,178.33365037,CREDITO,0,178.33365037,0.0,1.0,0.0,1.0,0,178.33365037,1
1,075d178871d8d48502bf1f54887e52fe,a53bb81bd0bba2ae2535bda7ea5a550c,2d8d34be7509a6b1262336d036fdb324,2021-09-12 18:31:58,c0b62f9046c83ea5543ea46a497a4d6e,5cfff960ea6d732c1ba3e63d24f3be52,35.66673007,CREDITO,0,35.66673007,0.0,1.0,0.0,1.0,0,35.66673007,1
2,075d178871d8d48502bf1f54887e52fe,79f893ea65c06fe2933f3847c88c272f,5eeb18254850b21af0a6bb2697913cd3,2021-09-12 18:31:56,872d10143fc0ac7d5de467806f6bef81,c97e63a92c82c7217b333635d75928ed,142.66692029,CREDITO,0,142.66692029,0.0,1.0,0.0,1.0,0,142.66692029,1
3,075d178871d8d48502bf1f54887e52fe,ce577223699dbdc119df2ab8a35457fe,5221a599856c0a0588ed9ffd150edd3b,2021-09-12 18:31:10,2e35cfe7860a480a93e1c83e99843579,fc09bdd00f283222d65eaff4d00a6594,8.32223701,CREDITO,0,8.32223701,0.0,1.0,0.0,1.0,0,8.32223701,1
4,075d178871d8d48502bf1f54887e52fe,0ce7ab2950e7a788a608f881aec0f8f0,971e55ef12d80ec070ea4f6750c8b892,2021-09-12 18:31:07,995bc89e4c4e00334f1f90c4a55f4729,213527e8ba94fcaf2f9378969f9f6abc,32.10005706,CREDITO,0,55.87787710,0.0,2.0,0.0,1.0,0,55.87787710,2


In [171]:
DATOS_TRANSACCIONALES_1['Flag']  = np.where(((DATOS_TRANSACCIONALES_1['suma_trx'] - DATOS_TRANSACCIONALES_1['suma_trx_debito'] - DATOS_TRANSACCIONALES_1['suma_trx_credito']) == (DATOS_TRANSACCIONALES_1['suma_trx_debito'] + DATOS_TRANSACCIONALES_1['suma_trx_credito'])) & (DATOS_TRANSACCIONALES_1['count_trx'] > 2)  & ((DATOS_TRANSACCIONALES_1['count_trx'] -DATOS_TRANSACCIONALES_1['cuenta_trx_debito']-DATOS_TRANSACCIONALES_1['cuenta_trx_credito']) != (DATOS_TRANSACCIONALES_1['cuenta_trx_debito']+DATOS_TRANSACCIONALES_1['cuenta_trx_credito']) ) ,1, 0)

In [172]:
DATOS_TRANSACCIONALES_1[DATOS_TRANSACCIONALES_1['Flag'] == 1]

,merchant_id,_id,subsidiary,transaction_date,account_number,user_id,transaction_amount,transaction_type,suma_trx,count_trx,suma_trx_debito,suma_trx_credito,cuenta_trx_debito,cuenta_trx_credito,count_cuentas_debito,count_cuentas_credito,Flag
41511,075d178871d8d48502bf1f54887e52fe,a49dd48255968c02ccf5df1911361f6d,029fd27c10647965a5632dc299e822f6,2021-11-26 11:43:01,a950d46f101464e4aa19d50bd9e04999,e70dc6f1046ccc6e401727528e026156,178.33365037,CREDITO,1545.55830320,3,0,772.77915160,0.0,2.0,0.0,1.0,1
41685,075d178871d8d48502bf1f54887e52fe,7a8507d1c870c36daa418e6e4607ae58,029fd27c10647965a5632dc299e822f6,2021-11-26 11:42:21,a950d46f101464e4aa19d50bd9e04999,e70dc6f1046ccc6e401727528e026156,594.44550123,CREDITO,1545.55830320,3,0,772.77915160,0.0,2.0,0.0,1.0,1
61509,075d178871d8d48502bf1f54887e52fe,1b944efe40168964561de89f805365cb,3458b243beebecf55605ca649b6b2ea5,2021-02-08 15:44:02,bab8942f54f0c7548c3981f5f3613130,3c3dcfdb0f86d8a1f7c2bb510e8167f3,475.55640098,CREDITO,951.11280196,3,0,475.55640098,0.0,1.0,0.0,1.0,1
86921,075d178871d8d48502bf1f54887e52fe,fa20e226b5f5d1964031c3ef50e2edd3,ba400763dcda307d2ba5d6f7adf8e9e6,2021-01-06 10:30:25,25f4652865c3bf127ecfeba99ae2a8b8,507439cf8822e615b4f542fb5f532001,594.44550123,CREDITO,1426.66920294,6,0,713.33460147,0.0,2.0,0.0,1.0,1
86935,075d178871d8d48502bf1f54887e52fe,601ab2f547afc0ff3b952be55b889fa6,ba400763dcda307d2ba5d6f7adf8e9e6,2021-01-06 10:31:08,25f4652865c3bf127ecfeba99ae2a8b8,507439cf8822e615b4f542fb5f532001,118.88910024,CREDITO,1426.66920294,6,0,713.33460147,0.0,2.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10741037,838a8fa992a4aa2fb5a0cf8b15b63755,a9dd260f9fe2a3bb213729a6f6be6d18,7b3098071d23be352bc6c85ba79e9228,2021-11-22 09:38:48,366626deedb2864f14b8f75daf7b1fe3,3de835b5703fd8a0918ed7487cad97c5,594.44550123,CREDITO,2258.89290468,3,0,1129.44645234,0.0,2.0,0.0,1.0,1
10741038,838a8fa992a4aa2fb5a0cf8b15b63755,9bac469c199d07e3f02db34a2dd1fe09,7b3098071d23be352bc6c85ba79e9228,2021-11-22 09:39:25,366626deedb2864f14b8f75daf7b1fe3,3de835b5703fd8a0918ed7487cad97c5,535.00095111,CREDITO,2258.89290468,3,0,1129.44645234,0.0,2.0,0.0,1.0,1
10747112,838a8fa992a4aa2fb5a0cf8b15b63755,f53ef0860da54d9a5e40c188ae33a327,7b3098071d23be352bc6c85ba79e9228,2021-11-13 12:49:21,793027427dc48fe490c4b719ab54879d,670f0b9a982f451d36c0c53897ffa633,594.44550123,CREDITO,2258.89290468,3,0,1129.44645234,0.0,2.0,0.0,1.0,1
10747113,838a8fa992a4aa2fb5a0cf8b15b63755,bb59065f62e013c815d1dab5bad3d432,7b3098071d23be352bc6c85ba79e9228,2021-11-13 12:49:52,793027427dc48fe490c4b719ab54879d,670f0b9a982f451d36c0c53897ffa633,535.00095111,CREDITO,2258.89290468,3,0,1129.44645234,0.0,2.0,0.0,1.0,1


Ajustar modelo

In [191]:
BASE_MODELO = DATOS_TRANSACCIONALES_1[['_id','transaction_amount','suma_trx', 'count_trx','suma_trx_debito','suma_trx_credito','cuenta_trx_debito','cuenta_trx_credito','count_cuentas_debito','count_cuentas_credito']]

In [192]:
BASE_MODELO = BASE_MODELO.set_index('_id')

In [188]:
DATOS_TRANSACCIONALES_1['Flag'] = DATOS_TRANSACCIONALES_1['Flag'].astype(str)

In [193]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_Y, test_Y = train_test_split(BASE_MODELO, DATOS_TRANSACCIONALES_1['Flag'], test_size = 0.25, random_state = 42,  stratify=  DATOS_TRANSACCIONALES_1['Flag'])

In [194]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(train_X, train_Y)

RandomForestClassifier()

In [196]:
predictions = rf.predict(test_X)

In [200]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
mat_confusion = confusion_matrix(
                    y_true    = test_Y,
                    y_pred    = predictions
                )

accuracy = accuracy_score(
            y_true    = test_Y,
            y_pred    = predictions,
            normalize = True
           )

print("Matriz de confusión")
print("-------------------")
print(mat_confusion)
print("")
print(f"El accuracy de test es: {100 * accuracy} %")

Matriz de confusión
-------------------
[[2688773      17]
 [     52     763]]

El accuracy de test es: 99.99743456752942 %


REGRESIÓN LOGISTICA

In [201]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [202]:
modelo = LogisticRegression(penalty='none')
modelo.fit(X = train_X, y = train_Y)

C:\Users\ytati\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\ytati\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(penalty='none')

In [203]:
predicciones = modelo.predict(X = test_X)

In [204]:
predicciones

array(['0', '0', '0', ..., '0', '0', '0'], dtype=object)

In [206]:
accuracy = accuracy_score(
            y_true    = test_Y,
            y_pred    = predicciones,
            normalize = True
           )
print("")
print(f"El accuracy de test es: {100*accuracy}%")


El accuracy de test es: 99.96958661216053%


# ORGANIZADO

In [139]:
CREDITO_1=DATOS_TRANSACCIONALES_1[DATOS_TRANSACCIONALES_1['transaction_type'] == 'CREDITO']
# DEBITO_1=DATOS_TRANSACCIONALES_1[DATOS_TRANSACCIONALES_1['transaction_type'] == 'DEBITO']

In [140]:
CREDITO_1["MAX_TIME"]=CREDITO_1.transaction_date+datetime.timedelta(days=1)
# CREDITO_1["MIN_TIME"]=CREDITO_1.transaction_date+datetime.timedelta(days=-1)
# CREDITO_1=CREDITO_1.rename(columns={"_id": "_id_CREDIT"})
CREDITO_1=CREDITO_1[['_id','user_id','account_number','transaction_date','MAX_TIME']]

C:\Users\ytati\AppData\Local\Temp\ipykernel_13876\2331624708.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CREDITO_1["MAX_TIME"]=CREDITO_1.transaction_date+datetime.timedelta(days=1)


In [141]:
FULL_DATA=DATOS_TRANSACCIONALES_1.merge(CREDITO_1,on=('user_id','account_number'),suffixes=("","_CREDIT"),how='left')
# FULL_DATA["INSIDE_BOUNDARIES"]=FULL_DATA.groupby(['_id']).apply(lambda x: 1 if (x['transaction_date'].max<x['MAX_TIME'].max) and (x['transaction_date'].max>x['MIN_TIME'].max )else 0)

In [142]:
# FULL_DATA_END.groupby(['user_id','account_number'])['merchant_id'].nunique().reset_index().sort_values(by='merchant_id',ascending=False)
FULL_DATA_END=FULL_DATA[(FULL_DATA['transaction_date']<=FULL_DATA['MAX_TIME']) & (FULL_DATA['transaction_date']>=FULL_DATA['transaction_date_CREDIT'])]

In [143]:
# DATOS_TRAIN_1=FULL_DATA_END.iloc[1:100000,:]
DATOS_TRANSACCIONALES_1_AFTER_GROUPED=FULL_DATA_END.groupby(['user_id','account_number','transaction_date_CREDIT'])
DATOS_TRANSACCIONALES_1_AFTER_GROUPED_END=DATOS_TRANSACCIONALES_1_AFTER_GROUPED['_id'].count()
DATOS_TRANSACCIONALES_1_AFTER_GROUPED_END=pd.DataFrame(DATOS_TRANSACCIONALES_1_AFTER_GROUPED_END).reset_index()

In [144]:
def count_credit(x):return list(x).count("CREDITO")
def count_debit(x):return list(x).count("DEBITO")

In [145]:
# DATOS_TRANSACCIONALES_1_AFTER_GROUPED_END['MAX_TIME']=DATOS_TRANSACCIONALES_1_AFTER_GROUPED['MAX_TIME'].transform(lambda x: np.nanmax(x))
# DATOS_TRANSACCIONALES_1_AFTER_GROUPED_END['MIN_TIME']=DATOS_TRANSACCIONALES_1_AFTER_GROUPED['MIN_TIME'].transform(lambda x: np.nanmin(x))
DATOS_TRANSACCIONALES_1_AFTER_GROUPED_END['CANTIDAD_CREDITO']=DATOS_TRANSACCIONALES_1_AFTER_GROUPED['transaction_type'].agg([("temp_fun",count_credit)]).reset_index().temp_fun
DATOS_TRANSACCIONALES_1_AFTER_GROUPED_END['CANTIDAD_DEBITO']=DATOS_TRANSACCIONALES_1_AFTER_GROUPED['transaction_type'].agg([("temp_fun",count_debit)]).reset_index().temp_fun
DATOS_TRANSACCIONALES_1_AFTER_GROUPED_END['MONTO_CREDITO']=DATOS_TRANSACCIONALES_1_AFTER_GROUPED.apply(lambda x:np.sum(list(compress(x['transaction_amount'],[(lambda y: True if y=="CREDITO" else False)(X) for X in x['transaction_type']])))).reset_index().iloc[:,3]
DATOS_TRANSACCIONALES_1_AFTER_GROUPED_END['MONTO_DEBITO']=DATOS_TRANSACCIONALES_1_AFTER_GROUPED.apply(lambda x:np.sum(list(compress(x['transaction_amount'],[(lambda y: True if y=="DEBITO" else False)(X) for X in x['transaction_type']])))).reset_index().iloc[:,3]
# DATOS_TRANSACCIONALES_1_AFTER_GROUPED_END['MONTO_CREDITO']=DATOS_TRANSACCIONALES_1_AFTER_GROUPED['transaction_type','transaction_amount'].transform(lambda x,y:list(compress(x,[(lambda z: True if z=="CREDITO" else False)(X) for X in y])).sum)
# DATOS_TRANSACCIONALES_1_AFTER_GROUPED_END['MONTO_CREDITO']=DATOS_TRANSACCIONALES_1_AFTER_GROUPED.agg({})

In [147]:
CASOS_END=DATOS_TRANSACCIONALES_1_AFTER_GROUPED_END[(DATOS_TRANSACCIONALES_1_AFTER_GROUPED_END['MONTO_CREDITO']==DATOS_TRANSACCIONALES_1_AFTER_GROUPED_END['MONTO_DEBITO'])& (DATOS_TRANSACCIONALES_1_AFTER_GROUPED_END['CANTIDAD_CREDITO']<2) & (DATOS_TRANSACCIONALES_1_AFTER_GROUPED_END['CANTIDAD_DEBITO']>1) ].sort_values(by='CANTIDAD_DEBITO',ascending=False)

In [162]:
FULL_CASOS_END=CASOS_END.loc[:,['user_id','account_number','transaction_date_CREDIT','CANTIDAD_CREDITO','CANTIDAD_DEBITO','MONTO_CREDITO','MONTO_DEBITO']].merge(FULL_DATA_END,on=('user_id','account_number','transaction_date_CREDIT'),how='inner')

In [171]:
FULL_CASOS_END[(FULL_CASOS_END['_id_CREDIT'] == '2deb9bee5c441490adaf04eba0a4fbb0')]#.sort_values(by='CANTIDAD_DEBITO',ascending=False)

,user_id,account_number,transaction_date_CREDIT,CANTIDAD_CREDITO,CANTIDAD_DEBITO,MONTO_CREDITO,MONTO_DEBITO,merchant_id,_id,subsidiary,transaction_date,transaction_amount,transaction_type,_id_CREDIT,MAX_TIME
427,f65cccf895d9faa686829c249d444cfc,6dc6a4a1de8fb43a749647ddf5602002,2021-04-06 10:39:37,1,2,594.44550123,594.44550123,075d178871d8d48502bf1f54887e52fe,2deb9bee5c441490adaf04eba0a4fbb0,a495d66c5e5b637c14fe8e1ab2ca9e92,2021-04-06 10:39:37,594.44550123,CREDITO,2deb9bee5c441490adaf04eba0a4fbb0,2021-04-07 10:39:37
428,f65cccf895d9faa686829c249d444cfc,6dc6a4a1de8fb43a749647ddf5602002,2021-04-06 10:39:37,1,2,594.44550123,594.44550123,817d18cd3c31e40e9bff0566baae7758,b40c21c17abaf25ba5a8bfa7de18713e,4baa10d32a94dbfe6e644fd0fb5d4ad5,2021-04-06 15:08:42,237.77820049,DEBITO,2deb9bee5c441490adaf04eba0a4fbb0,2021-04-07 10:39:37
429,f65cccf895d9faa686829c249d444cfc,6dc6a4a1de8fb43a749647ddf5602002,2021-04-06 10:39:37,1,2,594.44550123,594.44550123,817d18cd3c31e40e9bff0566baae7758,546d258f3f3ad35f69b7507cc0c3bca4,65c51d0a28402d1a208b0a6af4b67462,2021-04-06 20:56:21,356.66730074,DEBITO,2deb9bee5c441490adaf04eba0a4fbb0,2021-04-07 10:39:37
